In [19]:
import torch

In [20]:
torch.cuda.is_available()

True

In [21]:
import pandas as pd
df = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\qajokes1.1.2.csv")
df.head(5)

,Question,Answer,dirt,humor,kwid,meta
0,What's the best anti diarrheal prescription?,Mycheexarphlexin,0.0,0.2,BEST-ANTI-DIARRHEAL-PRESCRIPTION-MYCHEEXARPHLEXIN,NaN
1,What do you call a person who is outside a doo...,Matt,0.0,0.2,PERSON-OUTSIDE-DOOR-ARMS-LEGS-MATT,NaN
2,Which Star Trek character is a member of the m...,Jean-Luc Pickacard,0.0,0.2,STAR-TREK-CHARACTER-MEMBER-MAGIC-CIRCLE-JEANLU...,NaN
3,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe,0.0,0.2,DIFFERENCE-BULLET-HUMAN-BULLET-DOESNT-MISS-HAR...,NaN
4,Why was the Ethiopian baby crying?,He was having a mid-life crisis,0.0,0.2,ETHIOPIAN-BABY-CRYING-MIDLIFE-CRISIS,NaN


In [22]:
df = df.drop(columns=['kwid' , 'meta', 'dirt' , 'humor'])
df = df.dropna()
df = df.sample(frac=1).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75107 entries, 0 to 75106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  75107 non-null  object
 1   Answer    75107 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [23]:
df.sample(5)

,Question,Answer
29561,Why is there no life on Mars?,No WiFi...
58362,What does a gay guy say when he can't remember?,I faggot
71491,Who let the dogs out?,Chuck Norris let the dogs out...... and then r...
64865,What did the Indian man name his sandwich shop?,New Delhi :D
39677,What do you call it when an Italian has one ar...,A speech impediment.


In [24]:
#combine convokit dataset also
df2 = pd.read_csv(r"E:\Projects\encoder - Decoder - chatbot\Convokit_dataset.csv")
df2 = df2.drop(columns=['Unnamed: 0'])
df2 = df2.dropna()
df2 = df2.sample(frac=1).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   questions  221282 non-null  object
 1   answers    221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [25]:
#concatinating both datsets
# Rename columns in df2 to match df
df2 = df2.rename(columns={'questions': 'Question', 'answers': 'Answer'})

# Concatenate both datasets
dfn = pd.concat([df, df2], ignore_index=True)

dfn = dfn.sample(frac=1).reset_index(drop=True)

dfn.head()

,Question,Answer
0,MOM!,Make that sixty and I'm docking your allowance...
1,Name's Otis.,I know you?
2,"That was your idea, I wanted to...","Shit, Kid, you pulled your pistol... right out..."
3,Because - monster as I was - I was bound by La...,Kirsty Cotton. Yes. But ... if your soul was...
4,Know about it? You think the Board would do an...,You mean you know about this?


In [26]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296389 entries, 0 to 296388
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Question  296389 non-null  object
 1   Answer    296389 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


Tokenization

In [27]:
PAD_token = 0 #used for padding short sentences
SOS_token = 1 #start of sentence token
EOS_token = 2 #End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD" , SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 #count sos , eos , pad

    def addSentence(self , sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self , word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] +=1

In [28]:
import unicodedata
import re

# Define contractions dictionary
good_prefixes = {
    "i'm": "i am",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "that's": "that is",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
}

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = " ".join([good_prefixes[word] if word in good_prefixes else word for word in s.split()])
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()    
    return s

# Read query / response pairs and return a voc object
def readVocs(datafile , corpus_name):
    print("Reading lines......")
    
    # Read the file and split into lines
    lines = open(datafile , encoding='utf-8').read().strip().split('\n')

    #split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('<CoSe')] for l in lines]
    voc = Voc(corpus_name)
    return voc , pairs


# Returns true iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 15
def filterPair(p):
    #input sequences need to preserve the last word for EOS token 
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#using the functions defined above , return a populated voc object and pairs list
def loadPrepareData(corpus_name , datafile , save_dir):
    print("Start preparing training data .... ")
    voc , pairs = readVocs(datafile , corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words....")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words: ", voc.num_words)
    return voc , pairs

Batchwise input and data preparation

In [29]:
import itertools

PAD_token = 0  # used for padding short sentences
SOS_token = 1  # start of sentence token
EOS_token = 2  # End-of-sentence token

def indexesFromSentence(voc , sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l , fillvalue = PAD_token):
    return list(itertools.zip_longest(*l , fillvalue = fillvalue))

def binaryMatrix(l , value=PAD_token):
    m = []
    for i , seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# returns padded input sequence tensor and lengths 
def inputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar , lengths

# returns padded target sequence tensor, padding mask , and max target length
def outputVar(l , voc):
    indexes_batch = [indexesFromSentence(voc , sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar , mask , max_target_len

#returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
        print("Input sentence-",pair[0])
        print("Output sentece-",pair[1])
        print("-----------------------")
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


In [30]:
import random

# Create a vocabulary instance
voc = Voc('test')

# Create pairs from your existing dataframe and normalize them
print("Creating pairs from dataframe...")
pairs = []
for _, row in dfn.iterrows():
    question = normalizeString(row['Question'])
    answer = normalizeString(row['Answer'])
    pairs.append([question, answer])

# Build vocabulary from pairs
print("Building vocabulary...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])

# Example for validation
print("\nTesting with a small batch...")
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nBatch information:")
print("Input shape:", input_variable.shape)
print("Lengths:", lengths)
print("Output shape:", target_variable.shape)

Creating pairs from dataframe...
Building vocabulary...

Testing with a small batch...
Input sentence- good . you ve got the only truck in the valley that can make it up that damn jeep trail . so here s the plan you and heather go for help . get to the mountains . . .
Output sentece- no problem .
-----------------------
Input sentence- probably safe in the hands of the federal government .
Output sentece- alright . . . turn around . she does not get shit unless i get that money . where is it ?
-----------------------
Input sentence- what is the difference between camping and being homeless ?
Output sentece- marshmallows
-----------------------
Input sentence- do you know honduras johnny ?
Output sentece- mr . san pedro sula is from honduras .
-----------------------
Input sentence- a computer canna lie sir .
Output sentece- yet the data banks insist we fired twice . one computer is lying .
-----------------------

Batch information:
Input shape: torch.Size([41, 5])
Lengths: tensor([41,

In [31]:
def filterPair(voc, pair):
	# Return True if all words in both sentences are in vocab
	return all(word in voc.word2index for word in pair[0].split()) and \
		   all(word in voc.word2index for word in pair[1].split())

# Get valid pairs
valid_pairs = [pair for pair in pairs if filterPair(voc, pair)]

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(valid_pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("\nInput variable:")
print("input shape:", input_variable.shape)
print("lengths:", lengths)
print("\nTarget variable:")
print("Output shape:", target_variable.shape)
print("mask shape:", mask.shape)
print("max_target_len:", max_target_len)

Input sentence- ha ! what a load of crap . look at yourself caesar . you re a thug . you launder money for the mob . you rent women like you rented this apartment .
Output sentece- i saved you .
-----------------------
Input sentence- naah i am done . got to be on shift in a couple hours . are not you going in ?
Output sentece- have another one sounds like you need it .
-----------------------
Input sentence- what qualifies as a short stack of pancakes ?
Output sentece- i mean . feet is relatively short right yes okay cool . then i just ate a short stack .
-----------------------
Input sentence- what do black men hear most after sex ?
Output sentece- back to your cell inmate .
-----------------------
Input sentence- were you a drug addict ?
Output sentece- no .
-----------------------

Input variable:
input shape: torch.Size([35, 5])
lengths: tensor([35, 22, 10, 10,  7])

Target variable:
Output shape: torch.Size([21, 5])
mask shape: torch.Size([21, 5])
max_target_len: 21


Architecture

In [ ]:
import torch.nn as nn
class EncoderRNN(nn.Module):
    def __init__(self , hidden_size, embedding, n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding

#Initialize GRU ; THE input_size and hidden_size params are both set to 'hidden_size'ArithmeticError
#because our input size is a word embedding with number of features == hidden_size

        self.gru = nn.GRU(hidden_size,hidden_size,n_layers, dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths , hidden=None):
        
        #convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        print(f"Shape after embedding: {embedded.shape}")
        
        #pack padded batch of sequences for RNN modu;e
        #making sure that input_lengths stays on cpu
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu())
        print(f"Shape after packing: {packed.data.shape}")
        
        #forward pass through GRU
        outputs , hidden = self.gru(packed , hidden)
        print(f"Shape after GRU: {outputs.data.shape}")
        
        #unpack padding
        outputs , _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        print(f"Shape after unpacking: {outputs.shape}")
        
        #sum birectional GRU  outputs
        outputs = outputs[:,:,:self.hidden_size] + outputs[:,:,self.hidden_size:]
        print(f"Shape after summing bidirectional outputs: {outputs.shape}")
        
        #return output and final hidden state
        return outputs, hidden

Attention layer and decoder

attention layer

In [ ]:
import torch.nn.functional as F

# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)
    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
# Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
# Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

decoder

In [34]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
# Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
# Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.attn = Attn(attn_model, hidden_size)
def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

Loss function

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Training Model and Evaluation

In [37]:
import os

# Model configuration
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set training device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)

# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN('dot', embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

# Use CUDA if available
encoder = encoder.to(device)
decoder = decoder.to(device)

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.9
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 50
print_every = 100
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# Set model name and directory for saving
model_name = 'chatbot_model'
corpus_name = 'cornell_movie_dialogs'
save_dir = os.path.join("data", "save")

# Create save directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Set loadFilename to None as we're training from scratch
loadFilename = None



Building optimizers ...


In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
               embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
               print_every, save_every, clip, corpus_name, loadFilename):
    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                       for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    
    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                    decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(
                iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if iteration % save_every == 0:
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(
                encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder,
          embedding, encoder_optimizer, decoder_optimizer, batch_size, clip):
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # lengths = lengths.to(device) lengths are expected to be in cpu

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if teacher forcing should be used for this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    loss = 0
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_input = target_variable[t].view(1, -1)  # Next input is current target
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            _, topi = decoder_output.topk(1)  # [64, 1]
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss

    # Perform backpropagation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Starting Training!
Input sentence- you still do not see it do you charlie ? you are nothing but a small time charlie . you d love to keep me hustling for you huh ? wouldn t ya ? i mean a couple more years with me scuffling around them little towns and those back alleys . you might make yourself enough to get a little poolroom back in oakland . six tables and a handbook on the side . is that when you say goodbye to me charlie ?
Output sentece- you wanna come back on the road with me okay the money s yours . but if you wanna give it to minnesota fats . . . nothing doing . what do you say ?
-----------------------
Input sentence- angola . my mother was being a surgeon there . she volunteered . mozambique and mali too . . . . . .those days in those days . . .she was part of cuba s gift to the third world . she came from a bourgeois family with a house in verdado . . .we still have the house . we will go there . . .
Output sentece- were you born in havana ?
-----------------------
Input sen

RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor, but got 1D cuda:0 Long tensor